In [ ]:
import os
import json
import shutil

# Replace with your actual token
kaggle_api_dict = {"username":"","key":""}

# Write to kaggle.json
with open("/kaggle/working/kaggle.json", "w") as f:
    json.dump(kaggle_api_dict, f)


In [ ]:
os.makedirs("/root/.kaggle", exist_ok=True)
shutil.move("/kaggle/working/kaggle.json", "/root/.kaggle/kaggle.json")
os.chmod("/root/.kaggle/kaggle.json", 600)


In [ ]:
# from kaggle.api.kaggle_api_extended import KaggleApi

# api = KaggleApi()
# api.authenticate()


In [ ]:
pip install kaggle


Note: you may need to restart the kernel to use updated packages.


In [ ]:
!kaggle competitions download -c shl-intern-hiring-assessment


 98%|██████████████████████████████████████▏| 1.07G/1.10G [00:04<00:00, 309MB/s]
100%|███████████████████████████████████████| 1.10G/1.10G [00:04<00:00, 253MB/s]


In [ ]:
!unzip -q shl-intern-hiring-assessment.zip -d /kaggle/working/


In [ ]:
pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 16.8 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.8 MB/s eta 0:00:000:00:0100:01
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803405 sha256=1a51cec1c28e243ebeaa266f4444c2c0f2f26fec6b4dd3cca6fbc41aa61350d4
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install openai-whisper language-tool-python spacy textstat librosa pandas tqdm spacy



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 70.1 MB/s eta 0:00:00


In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 51.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import os
import whisper
import pandas as pd
import language_tool_python
import spacy
import textstat
import librosa
from tqdm import tqdm

# Load models
whisper_model = whisper.load_model("turbo")
tool = language_tool_python.LanguageTool('en-US')
nlp = spacy.load("en_core_web_sm")

def transcribe_audio(audio_path):
    result = whisper_model.transcribe(audio_path)
    return result["text"]

def grammar_error_rate(text):
    matches = tool.check(text)
    word_count = len(text.split())
    return len(matches) / word_count * 100 if word_count > 0 else 0

def syntactic_complexity(text):
    doc = nlp(text)
    sentence_lengths = [len(sent.text.split()) for sent in doc.sents]
    avg_sentence_length = sum(sentence_lengths) / len(sentence_lengths) if sentence_lengths else 0
    complex_sentences = sum(1 for sent in doc.sents if any(tok.dep_ == "advcl" or tok.dep_ == "ccomp" for tok in sent))
    return avg_sentence_length, complex_sentences

def speaking_rate(audio_path, transcript):
    y, sr = librosa.load(audio_path)
    duration = librosa.get_duration(y=y, sr=sr)
    word_count = len(transcript.split())
    return word_count / duration if duration > 0 else 0

def process_dataset(folder_path, csv_path, is_train=True):
    df = pd.read_csv(csv_path)
    features = []

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
        audio_id = row['filename']
        audio_file = os.path.join(folder_path, f"{audio_id}")

        try:
            transcript = transcribe_audio(audio_file)
            ge_rate = grammar_error_rate(transcript)
            avg_sent_len, complex_sent_count = syntactic_complexity(transcript)
            rate = speaking_rate(audio_file, transcript)

            data = {
                "id": audio_id,
                "normalized_grammar_errors": ge_rate,
                "avg_sentence_length": avg_sent_len,
                "complex_sentence_count": complex_sent_count,
                "speaking_rate": rate
            }

            if is_train:
                data["score"] = row["label"]

            features.append(data)
        except Exception as e:
            print(f"Error processing {audio_id}: {e}")

    return pd.DataFrame(features)

# Paths
train_folder = "audios_train"
test_folder = "audios_test"
train_csv_path = "train.csv"
test_csv_path = "test.csv"


train_features = process_dataset(train_folder, train_csv_path, is_train=True)
test_features = process_dataset(test_folder, test_csv_path, is_train=False)


train_features.to_csv("train_features.csv", index=False)
test_features.to_csv("test_features.csv", index=False)

print("Feature extraction completed and CSV files saved.")


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
/usr/local/li

Feature extraction completed and CSV files saved.


In [ ]:
import whisper
import os
import pandas as pd


model = whisper.load_model("turbo")


audio_folder = "audios_test"
output_csv = "transcriptions_test.csv"

results = []

for filename in os.listdir(audio_folder):
    if filename.endswith((".mp3", ".wav", ".m4a")):
        audio_path = os.path.join(audio_folder, filename)
        print(f"Transcribing {filename}...")

        result = model.transcribe(audio_path)
        transcription = result["text"]

        results.append({
            "filename": filename,
            "transcription": transcription
        })

df = pd.DataFrame(results)
df.to_csv(output_csv, index=False)

print(f"\n✅ Transcriptions saved to {output_csv}")


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Transcribing audio_550.wav...
Transcribing audio_820.wav...
Transcribing audio_113.wav...
Transcribing audio_735.wav...
Transcribing audio_1280.wav...
Transcribing audio_135.wav...
Transcribing audio_1217.wav...
Transcribing audio_1035.wav...
Transcribing audio_1289.wav...
Transcribing audio_857.wav...
Transcribing audio_300.wav...
Transcribing audio_137.wav...
Transcribing audio_21.wav...
Transcribing audio_391.wav...
Transcribing audio_1315.wav...
Transcribing audio_1026.wav...
Transcribing audio_437.wav...
Transcribing audio_109.wav...
Transcribing audio_499.wav...
Transcribing audio_1173.wav...
Transcribing audio_604.wav...
Transcribing audio_698.wav...
Transcribing audio_287.wav...
Transcribing audio_276.wav...
Transcribing audio_543.wav...
Transcribing audio_920.wav...
Transcribing audio_1267.wav...
Transcribing audio_512.wav...
Transcribing audio_348.wav...
Transcribing audio_656.wav...
Transcribing audio_386.wav...
Transcribing audio_1293.wav...
Transcribing audio_1240.wav...
T

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score
import string
import nltk
import re

nltk.download("punkt")
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

transcriptions_df = pd.read_csv("transcriptions.csv")
scores_df = pd.read_csv("train.csv")

df = pd.merge(transcriptions_df, scores_df, on="filename")

# Preprocessing Function

stop_words = set(stopwords.words("english"))

def preprocess(text):
    text = text.lower()
    text = re.sub(r"\d+", "", text)  # remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stop_words and len(w) > 1]
    return " ".join(tokens)

df["clean_text"] = df["transcription"].apply(preprocess)


# Feature Extraction (TF-IDF)

vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1,2))
X = vectorizer.fit_transform(df["clean_text"]).toarray()
y = df["label"].values


# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Train XGBoost Regressor
model = make_pipeline(StandardScaler(), XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42))
model.fit(X, y)

# Predict & Map to Valid Scores
valid_scores = np.array([0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0])

def map_to_closest_valid(y_preds, valid_scores):
    return np.array([valid_scores[np.abs(valid_scores - y).argmin()] for y in y_preds])

y_pred = model.predict(X_test)
y_pred_mapped = map_to_closest_valid(y_pred, valid_scores)
y_test_mapped = map_to_closest_valid(y_test, valid_scores)


# Evaluation
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
# accuracy = accuracy_score(y_test_mapped, y_pred_mapped)

print(f"R² Score: {r2:.3f}")
print(f"Mean Absolute Error: {mae:.3f}")
print(f"Mean Squared Error: {mse:.3f}")
print(f"Mapped Accuracy: {accuracy * 100:.2f}%")

joblib.dump(model, "xgboost_model.joblib")
joblib.dump(vectorizer, "tfidf_vectorizer.joblib")
print("✅ Model and vectorizer saved.")


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
R² Score: 0.832
Mean Absolute Error: 0.360
Mean Squared Error: 0.208
Mapped Accuracy: 86.52%
✅ Model and vectorizer saved.


In [ ]:

test_df = pd.read_csv("transcriptions_test.csv")


test_df["clean_text"] = test_df["transcription"].apply(preprocess)


X_test_new = vectorizer.transform(test_df["clean_text"]).toarray()


y_pred_new = model.predict(X_test_new)


y_pred_mapped_new = map_to_closest_valid(y_pred_new, valid_scores)


test_df["label"] = y_pred_mapped_new


test_df[["filename", "label"]].to_csv("predicted_scores.csv", index=False)

print("✅ Predictions saved to 'predicted_scores.csv'")


✅ Predictions saved to 'predicted_scores.csv'
